In [3]:
import torch  
from torch import nn

## 一、查询GPU的数量

In [17]:
torch.cuda.device_count()

1

In [19]:
def try_all_gpus(): #@save  
    """返回所有可用的GPU,如果没有GPU,则返回[cpu(),]"""  
    devices = [torch.device(f'cuda:{i}')  for i in range(torch.cuda.device_count())]  
    return devices if devices else [torch.device('cpu')]

In [21]:
try_all_gpus()

[device(type='cuda', index=0)]

In [23]:
def try_gpu(i=0): #@save  
    """如果存在,则返回gpu(i),否则返回cpu()"""  
    if torch.cuda.device_count() >= i + 1:  
        return torch.device(f'cuda:{i}')  
    return torch.device('cpu')

In [69]:
try_gpu()

device(type='cuda', index=0)

## 二、Tensor with GPU

In [28]:
x = torch.tensor([1, 2, 3])

1. 查询张量所在设备

In [31]:
x.device

device(type='cpu')

2. 将张量存储在GPU上

In [34]:
X = torch.ones(2, 3, device=try_gpu())
X.device

device(type='cuda', index=0)

In [40]:
Y = torch.rand(2, 3, device=try_gpu(1))  
Y.device

device(type='cpu')

没有俩个GPU，所以Y存放在CPU上

3. 复制

In [55]:
X+Y

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

此时，X和Y不在同一个设备上，无法进行运算，接下来，将Y复制到X的设备上。

In [58]:
Z= Y.cuda(0)
print(Y)
print(Z)

tensor([[0.9922, 0.6661, 0.9628],
        [0.1866, 0.9670, 0.7843]])
tensor([[0.9922, 0.6661, 0.9628],
        [0.1866, 0.9670, 0.7843]], device='cuda:0')


现在Y和X都在GPU上，可以进行运算

In [61]:
X+Z

tensor([[1.9922, 1.6661, 1.9628],
        [1.1866, 1.9670, 1.7843]], device='cuda:0')

## 三、神经网络与GPU

In [64]:
net = nn.Sequential(nn.Linear(3, 1))  
net = net.to(device=try_gpu())
net(X)

tensor([[-1.0294],
        [-1.0294]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [66]:
net[0].weight.data.device

device(type='cuda', index=0)

模型的参数也存放在一个设备上